In [ ]:
%matplotlib inline
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def get_year_data(year):
    FILE = '../Сводная вегетация.xlsx'
    SHEET = year

    df = pd.read_excel(FILE, sheetname=SHEET, header=1)

    NDVI_COLUMNS = [col for col in df.columns if 'неделя' in col]
    CULTURE_COLUMN = 'Культура ' + SHEET
    YIELD_COLUMN = 'Урожайность, т./га.'


    #df_cultures = df.groupby(df[CULTURE_COLUMN]).mean()
    df_cultures = df[df[YIELD_COLUMN].notnull()]#.reset_index()]
    df_cultures = df_cultures[df_cultures[CULTURE_COLUMN]=='Подсолнечник']
    df_cultures = df_cultures.reset_index()
    yield_by_culture = df_cultures[YIELD_COLUMN]
    culture = df_cultures[CULTURE_COLUMN]

    df_cultures = df_cultures[NDVI_COLUMNS]

    df_cultures = df_cultures.apply(lambda x: x.interpolate(method='linear'), axis=1)
    df_cultures = df_cultures.apply(lambda x: x.fillna(x[x.first_valid_index()]), axis=1)
    return df_cultures, yield_by_culture

In [ ]:
ndvi_2012, yield_2012 = get_year_data('2012')
ndvi_2012 = ndvi_2012.drop('неделя 53', 1)
ndvi_2013, yield_2013 = get_year_data('2013')
ndvi_2014, yield_2014 = get_year_data('2014')
ndvi_2015, yield_2015 = get_year_data('2015')

In [ ]:
frames = [ndvi_2012,ndvi_2013,ndvi_2014]
join_ndvi = pd.concat(frames)
#print(join_ndvi.info())
frames = [yield_2012,yield_2013,yield_2014]
join_yield = pd.concat(frames)
#print(join_yield)

In [ ]:
train_X = join_ndvi
train_y = join_yield

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(train_X,train_y)

In [ ]:
predictions = lm.predict(join_ndvi)
print(predictions)

In [ ]:
model.score(ndvi_2015,yield_2015)

In [ ]:
from sklearn import svm
from sklearn import metrics

X = join_ndvi
y = join_yield
model = svm.SVR(epsilon=0.001)
model.fit(X, y)


In [ ]:
pred = model.predict(join_ndvi)
print(pred)
pred = model.predict(ndvi_2015)
print(pred)
model.score(ndvi_2015,yield_2015)